## Youtube 스크립트 텍스트 전처리

In [41]:
import pandas as pd
import numpy as np
from hanspell import spell_checker # 500글자 이상의 문장은 검사 불가
import re
import warnings 
warnings.filterwarnings('ignore')

from konlpy.tag import Okt
okt = Okt()

In [2]:
path = input('절대 경로 입력 : ')

절대 경로 입력 :  C:\Users\j\stock_predict\Sentiment_Analysis\data\전처리 전 데이터\df_youtube_삼프로LG화학.csv


In [110]:
df1 = pd.read_csv(path)

In [111]:
df1.text

0      으[음악]tera term[음악]으 으 으 으 으[음악]이름은 하나인데 직업은 서먹...
1      구독과 좋아요는 사람입니다자 염치 염치 영차 차 우리 염성 안 차장에 보시겠습니다 ...
2      좋아요 구독 눌러주세요자 그럼 오늘 첫 번째 뉴스 부터 시작을 해 봐야죠먼저 뭐 미...
3      자 그러면 오늘 노려 일이니까요 발칙하게 한번 가보겠습니다뭐 리토의 방송에 건수를 ...
4      산업을 이해하고 기업을 이해해야 돈의 흐름이 보입니다리서치 명과 메리츠 증권과 3%...
                             ...                        
502    자 그러면 시간이 달했습니다 우리염불 이염 20 이상이면 경우 아침개장 상황 살펴보...
503    자 그러면 오늘 금요일 아침 9월10일 아침 증시 개장 은 어떻게 될것인지 우리 염...
504    4자 오늘 9월 9일 목요일 전혀아유진 자산운용의 김 타기 산이 모시고실제 에 돈을...
505    자 오늘 저희가 함께 이야기 나눌 분은 요 예 gs 자산운용예전에 그런 노래가 하나...
506    으[음악]으 으[음악]on으[박수][음악]기피하는 경제의 컨텐츠 오나 10나이브 져...
Name: text, Length: 507, dtype: object

In [112]:
# 데이터 전처리

df1.text = df1.text.str.replace('[으*]','')
df1.text = df1.text.str.replace('[\[\]]','')
df1.text = df1.text.str.replace('음악','')
df1.text = df1.text.str.replace('자','')
df1.text = df1.text.str.replace('[a-zA-Z]','')
df1.text = df1.text.str.replace('[구독*]','')
df1.text = df1.text.str.replace('[좋아요]','')
df1.text = df1.text.str.replace('[\s]','')
df1.text = df1.text.str.strip() # 앞뒤 공백 제거

df1.text 

0      이름은하나인데직업은서먹이라는말들어봤어화직업이소노계란이크게가능해본절본섬불운박극혐부부지...
1      과는사람입니다염치염치영차차우리염성안차장에보시겠습니다어서오십시오어서오세예어제밤새리포트...
2      눌러주세그럼오늘첫번째뉴스부터시작을해봐야죠먼저뭐미연준얘기근데뭐당연히매실얘기를먼저해야할...
3      그러면오늘노려일이니까발칙하게한번가보겠습니다뭐리토의방송에건수를기님어서오십시오이안녕하세...
4      산업을이해하고기업을이해해야돈의흐름이보입니다리서치명과메리츠증권과3%가함께야심차게준비한...
                             ...                        
502    그러면시간이달했습니다우리염불이염20이상이면경우침개장상황살펴보겠습니다베스트투증권의염승...
503    그러면오늘금일침9월10일침증시개장은어떻게될것인지우리염불베스트투증권의염승환이사님과함께...
504    4오늘9월9일목일전혀유진산운용의김타기산이모시고실제에돈을본편이좀그런가에는굴리는문들은어...
505    오늘저희가함께이야기나눌분은예산운용예전에그런노래가하나있었죠성은김이름은뒤했어그리고누가불...
506    박수기피하는경제의컨텐츠오나10나이브져용지힘도원입니다4장4예언하신라이브9월12일금일입...
Name: text, Length: 507, dtype: object

In [119]:
# 사전 로딩

import codecs

positive = [] 
negative = []
posneg = []
dic = [('./KOSELF_pos.txt', './KOSELF_neg.txt'),
       ("./positive_words_self.txt","./negative_words_self.txt"),
      ('./sum_pos.txt','./sum_neg.txt')]

i = int(input('koself : 1, blog : 2, sum : 3'))-1


pos = codecs.open(dic[i][0], 'rb', encoding='UTF-8') 

while True: 
    line = pos.readline() 
    line = line.replace('\n', '')
    line = line.replace('\r', '')
    
    if not line: break
        
    positive.append(line) 
    posneg.append(line) 
        
pos.close() 

neg = codecs.open(dic[i][1], 'rb', encoding='UTF-8') 
while True: 
    line = neg.readline() 
    line = line.replace('\n', '')
    line = line.replace('\r', '')
    
    if not line: break
    
    negative.append(line) 
    posneg.append(line) 
    
neg.close()

koself : 1, blog : 2, sum : 3 3


In [113]:
# 불용어
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

In [114]:
df1['token'] = ''

In [125]:
# test 10개만 빠르게 수행
df2 = df1[:100]

In [135]:
# 토큰화
for i in range(len(df2)):
    temp = []
    checked = []
    
    temp = okt.nouns(df2['text'].iloc[i])
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    temp = [word for word in temp if len(word) > 1] # 불용어 제거
#     print(i,'번 토큰화, 불용어 제거 완료')
    
    temp = list(set(temp))
#     print(i, ' 중복단어 제거')
    for check in temp:
        aa = spell_checker.check(check).checked
        checked.append(aa)
#     print(i,'- spell check')
    
    df2['token'].iloc[i] = checked
    print(i,'- 완료')

0 - 완료
1 - 완료
2 - 완료
3 - 완료
4 - 완료
5 - 완료
6 - 완료


ConnectionError: ('Connection aborted.', TimeoutError(10060, '연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다', None, 10060, None))

In [ ]:
j = 0
label = [0] * len(df2)

my_dic = {"text":[], "label":label}
texts = df2.token


for df in range(len(df2)):    
    txt_data = ','.join(texts[df]).replace(',', ' ')
    
    my_dic['text'].append(txt_data) 

    for i in range(len(posneg)): 
        posflag = False 
        negflag = False 

        if i < (len(positive)-1): 
            if txt_data.find(posneg[i]) != -1: 
                posflag = True 
                print(i, "positive?","테스트 : ",txt_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i) 
                break

        if i > (len(positive)-2): 
            if txt_data.find(posneg[i]) != -1: 
                negflag = True 
                print(i, "negative?","테스트 : ",txt_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i) 
                break

    if posflag == True: 
        label[j] = 1 
        # print("positive", j) 

    elif negflag == True: 
        label[j] = -1 
        # print("negative", j) 

    elif negflag == False and posflag == False: 
        label[j] = 0 
        # print("objective", j) 

    j = j + 1
                
my_dic['label'] = label
my_df = pd.DataFrame(my_dic)

In [ ]:
my_df

In [ ]:
my_df.label.value_counts()